In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
file_path = os.listdir("rbp_samplealiquot")

rbp = []
for i in file_path:
    RBP_output = pd.read_csv(f'rbp_samplealiquot/{i}')
    rbp.append(
        [RBP_output.iloc[8, 1], RBP_output.iloc[7, 1], RBP_output.iloc[0, 1]])

In [3]:
key = pd.DataFrame(rbp, columns=['RBP', 'SRP', 'Date'])
key['Date'] = pd.to_datetime(key['Date'])

## All Negative Plates

In [4]:
master = pd.read_pickle(
    '/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_call.pickle')

master = master[master['pos_tube_rack'].str.match(r'.*?(A6|B5)') == False]
master = master[master['run_sample_id'].str.startswith(('G', 'H'), na=False)]
master = master[~master['run_sample_id'].str.startswith('Ht', na=False)]

master['RBP'] = master['pos_tube_rack'].str.extract(r'(.*?):.*')

master = master.merge(key, left_on='RBP', right_on='RBP', how='left')
master.dropna(subset=['SRP', 'Date'], inplace=True)

master['total_not_detected'] = master['call'].map({'not_detected': 1,
                                                   'detected': 0,
                                                   'no_call': 0})
master['num_of_samples'] = master['sample_type'].map({'Sample': 1})

srp = master.groupby('RBP', sort=False).agg({'num_of_samples': 'sum',
                                                'total_not_detected': 'sum',
                                                'SRP': 'max',
                                                'Date': 'max'})

srp = srp[(srp['num_of_samples'] == srp['total_not_detected']) & (srp['total_not_detected'] > 70)]\
    .reset_index().sort_values(['Date', 'SRP'], ascending=[False, True])

In [5]:
srp

,RBP,num_of_samples,total_not_detected,SRP,Date
156,RBP201022A186,92,92,SRP200910A094,2020-12-03
157,RBP201022A245,92,92,SRP201026A125,2020-12-03
154,RBP201022A106,92,92,SRP201026A063,2020-12-01
153,RBP201022A087,92,92,SRP201026A064,2020-12-01
152,RBP201022A233,78,78,SRP201026A161,2020-12-01
...,...,...,...,...,...
168,RBP200702A081,92,92,SRP200728A027,2020-08-18
167,RBP200702A054,92,92,SRP200728A029,2020-08-18
99,RBP200727A040,72,72,SRP200728A084,2020-08-11
98,RBP200727A074,82,82,SRP200728A107,2020-08-11


#### SRP with Run Sample ID for PQ

In [6]:
SRP = ['SRP200910A107', 'SRP200910A104', 'SRP200910A210', 'SRP200910A198',
       'SRP200910A204', 'SRP200910A208', 'SRP200910A203', 'SRP200910A133',
       'SRP200901A344', 'SRP200910A170', 'SRP200910A185', 'SRP200910A175']

In [7]:
master[master['SRP'].isin(SRP)].sort_values(by=['SRP', 'RBP']).loc[:, ['SRP', 'run_sample_id']]

,SRP,run_sample_id
50263,SRP200901A344,HBR0723
50264,SRP200901A344,HBT0053
50265,SRP200901A344,HBJ0816
50266,SRP200901A344,HBQ0211
50267,SRP200901A344,HBT0389
...,...,...
62504,SRP200910A210,HBQ0872
62505,SRP200910A210,HBK1148
62506,SRP200910A210,HBT0901
62507,SRP200910A210,HBX0595


## >30% Positive

In [8]:
master = pd.read_pickle(
    '/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_call.pickle')
salesforce = pd.read_csv(
    '/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_tat.csv').fillna(0)

master = master[master['pos_tube_rack'].str.match(r'.*?(A6|B5)') == False]
master = master[master['run_sample_id'].str.startswith(('G', 'H'), na=False)]
master = master[~master['run_sample_id'].str.startswith('Ht', na=False)]

master['RBP'] = master['pos_tube_rack'].str.extract(r'(.*?):.*')

master = master.merge(key, left_on='RBP', right_on='RBP', how='left')
master.dropna(subset=['SRP', 'Date'], inplace=True)

master['total_detected'] = master['call'].map({'not_detected': 0,
                                               'detected': 1,
                                               'no_call': 0})
master['num_of_samples'] = master['sample_type'].map({'Sample': 1})

master['0.01-1'] = ((master['median_covid_ratio'] >= 0.01)
                    & (master['median_covid_ratio'] <= 1)).astype('int')
master['1-10'] = ((master['median_covid_ratio'] >= 1) &
                  (master['median_covid_ratio'] <= 10)).astype('int')
master['10-100'] = ((master['median_covid_ratio'] >= 10) &
                    (master['median_covid_ratio'] <= 100)).astype('int')
master['>100'] = (master['median_covid_ratio'] >= 100).astype('int')

binning = master.groupby('RBP', sort=False).agg({'num_of_samples': 'sum',
                                                 'total_detected': 'sum',
                                                 'SRP': 'max',
                                                 'Date': 'max',
                                                 '0.01-1': 'sum',
                                                 '1-10': 'sum',
                                                 '10-100': 'sum',
                                                 '>100': 'sum'})

binning = binning[(binning['total_detected'] > 10) & (binning['Date'] > '2020-10-01')
                  ].reset_index().sort_values(['Date', 'SRP'], ascending=[False, True])
binning['negative'] = binning['num_of_samples'] - binning['total_detected'] 

In [9]:
master.iloc[:, 25:].sum()

total_detected     1215
num_of_samples    57549
0.01-1              451
1-10                150
10-100              119
>100                495
dtype: int64

In [10]:
binning

,RBP,num_of_samples,total_detected,SRP,Date,0.01-1,1-10,10-100,>100,negative
4,RBP201112A183,92,26,SRP201026A297,2020-12-05,7,2,2,15,66
3,RBP201112A181,92,14,SRP201026A298,2020-12-05,3,1,1,9,78
5,RBP201112A184,92,29,SRP201026A312,2020-12-05,4,4,0,21,63
13,RBP201112A068,92,33,SRP200910A068,2020-12-03,3,6,9,15,59
12,RBP201112A067,92,15,SRP200910A069,2020-12-03,4,3,1,7,77
14,RBP201112A069,92,30,SRP200910A071,2020-12-03,7,5,2,16,62
9,RBP201022A182,92,16,SRP200910A095,2020-12-01,4,0,3,9,76
10,RBP201022A107,92,11,SRP201026A050,2020-12-01,2,0,0,9,81
8,RBP201022A181,92,16,SRP201026A061,2020-12-01,8,2,0,6,76
11,RBP201022A234,92,22,SRP201026A182,2020-12-01,7,4,2,9,70


In [11]:
RBP = binning.RBP.unique()
binning.sum()['total_detected']

434

In [12]:
only_pos = master[(master['total_detected'] == 1) & (master['Date'] > '2020-10-01') & (master['RBP'].isin(RBP))
                  ].sort_values(by=['run_sample_id', 'Date'], ascending=False)
only_pos = only_pos.merge(
    salesforce, left_on='run_sample_id', right_on='GH Sample ID', how='left')
only_pos = only_pos.iloc[:, [0, 1, 9, 23, 24, 25, 37]].sort_values(
    ['Date', 'SRP'], ascending=[False, True])
# only_pos.to_csv('retain_plates.csv', index=False)

In [13]:
len(only_pos)

434